# Просмотр метаданных Parquet файла

Ноутбук позволяет "разобраться" с Parquet файлами - посмотреть на структуру конкретного файла (на его метаданные).

Задача "прочитать" данные файла в ноутбуке не решается (как минимум, данных может быть много - используйте соответствующие инструменты, например, spark, pyarrow, duckdb)

Структура ноутбука повторяет структуру parquet файла - можно "прыгнуть" сразу в ту его часть, которая представляет интерес (воспользуйтесь возможностями "свертки" разделов ноутбука или содержанием).

**не забудьте проимпортировать" нужную функциональность (см. Imports), там же содержатся минимальные комментарии по технике.

# Imports

Для чтения мета-данных используется `thrift`

    python -m pip install thrift

Типы данных были сгенерены из текущей (на момент создания ноутбука) версии thrift спецификации parquet. Их можно просто использовать - спецификация parquet меняется не так часто, устанавливать для этого что бы то ни было не нужно.

Модуль ниже содержит минимальный набор нужных функций, облегчающих работу с метаданными

In [2]:
import pq_thrift_utils as pqu

Импортируем типы данных, созданные по thrift спецификации.

In [1]:
import ttypes as tt

Ну уж раз мы были здесь и что-то делали - давайте сразу определимся с тем файлом, который будем анализировать

In [3]:
PQ_FILE = "/home/mk/mk_win/Documents/28.Arrow/data/tpcds_10g.parquet/store_sales.parquet"

# Введение: как устроен Parquet файл

![](parquet_overview.gif)

Parquet файл - это последовательность следующих разделов:

* `"PAR1"`: magic в идеологии Linux (по первым нескольким байтам файла в Линуксе можно понять - что это за файл)
* `row_group[]`: список chunk-ов данных файла, chunk - некоторое количество строк (количество строк зависит от данных)
    * каждая колонока представлена в row_group в виде списка страниц (Page)
    * количество страниц в пределах row_group для разных колонок одинаково 
* (опционально) индексы: структуры данных, позволяющие понять, какие "страницы" можно не читать, исходя из мин-макс значений колонки в странице
    * эти индексы, а также bloom фильтры также организованы в страницы (Page) 
* (опционально) bloom фильтры: структуры данных, позволяющие понять, можно ли не читать row_group-у, исходя из значения колонки
* `footer`: метаданные файла (FileMetaData)
* `"PAR1"`: еще раз magic

Разделы и их части никак не разделены - следуют друг за другом, для доступа к разделу или его части используются абсолютные смещения в файле.

Структуры данных мета-данных файла - результат записи в файл thrift объектов (используется TCompactProtocol). Это подразумевает наличие "маркеров" окончания структур данных, маркер включен в длину структуры, более подробно с этим разберемся при знакомстве со страницами.

В этом ноутбуке разделы расположены в том же порядке и имеют схожие называния.

Для работы с parquet файлом "читатель" должен сначала прочитать `footer` (мета-данные), после чего может параллельно и независимо друг от друга обрабатывать `row_group-ы`.

Для анализа мета-данных также нужно начать с чтения мета-данных (см. соответствующий раздел), код других разделов предполагает, что мета-данные файла (объект FileMetaData) содержатся в переменной MD.

# Инструменталка

## decodeObject()

Основная функция создания thrift объектов из файла

In [4]:
help(pqu.decodeObject)

Help on function decodeObject in module pq_thrift_utils:

decodeObject(filePath, offset, obj, nBytes=10000)
    читает объект из thrift-файла filePath (предполагая использование TCompactProtocol)
    * offset: смещение объекта, если отрицательное - от конца файла
    * obj: тип возвращаемого объекта (в thrift спецификациях у всех объектов конструктор можно вызвать без параметров)
    * nBytes: как правило, в этом протоколе структуры заканчиваются маркером, поэтому можно читать больше, чем нужно

    Возвращает объект заданного типа.



## getLevelStr()

Функция для преобразования объекта в строку (оставляет только элементы первого уровня, "не лезет" вглубь)

In [5]:
help(pqu.getLevelStr)

Help on function getLevelStr in module pq_thrift_utils:

getLevelStr(what)
    Возвращает строку с одним "уровнем" детализации атрибутов объекта:

    * для атрибутов объекта типа list выводится длина списка
        * чтобы вывести содержимое списка - используем ее же
    * значения скалярных атрибутов выводятся (после двоеточия)
    * атрибуты, являющиеся объектами, помечены суффиксом "#"
    * атрибуты-методы - суффиксом "()"



## showExtra()

Бывает, что разбираясь, что-то хочется записать - делаю это в yaml файле ('extra_docs.yaml' - см. код). Для вывода использую `showExtra()`

In [6]:
help(pqu.showExtra)

Help on function showExtra in module pq_thrift_utils:

showExtra(obj)
    Выводит значение атрибута extra класса obj в виде Markdown в юпитере
    триммит слева пробелы



## На примере объекта FileMetaData

In [7]:
fLen = pqu.getFooterLen(PQ_FILE)
MD = pqu.decodeObject(PQ_FILE,-(8+fLen),tt.FileMetaData,fLen)

Комментарии к коду выше:

* длина footer-а содержится в конце файла, чтобы каждый раз не вспоминать, сделана функция, читающая его длину
    * в-принципе, она (длина) не нужна, считается и без нее... но она есть, воспользуемся 
* с помощью функции `decodeObject()` создал объект (в данном случае - `FileMetaData`) из соответствующей части parquet файла

С помощью функции `getLevelStr()` удобно смотреть атрибуты объекта (не загромождая лишними деталями - сравните с просто `print(MD)`)

In [8]:
print(pqu.getLevelStr(MD))

First level elements of <class 'ttypes.FileMetaData'>
column_orders: None
created_by: DuckDB version v1.3.2 (build 0b83e5d2f6)
encryption_algorithm: None
footer_signing_key_metadata: None
key_value_metadata: None
num_rows: 28800991
read()
row_groups[235]
schema[24]
thrift_spec#
validate()
version: 1
write()


Для наиболее важных объектов метаданных parquet файла добавлены дополнительные описания, посмотреть их можно с помощью функции `showExtra()`:

In [9]:
pqu.showExtra(MD)

Дополнительная информация (см. также help()):

* **num_rows**: количество строк в файле
* длина списка **row_groups** говорит нам о количестве row group в файле
* **schema**: схема данных (см. ниже)
* **key_value_metadata**: список метаданных (пар ключ-значение) на уровне файла


Если комментариев нет - так и "скажет"

In [ ]:
pqu.showExtra(MD.thrift_spec)

Пример вывода содержимого атрибута-списка:

In [ ]:
print(pqu.getLevelStr(MD.schema))

## help(), docstrings

Помочь разобраться с "увиденным" может документация (собственно, исходный thrift файл) - функция `help()` выведет немного лишнего:

In [10]:
help(MD)

Help on FileMetaData in module ttypes object:

class FileMetaData(builtins.object)
 |  FileMetaData(version=None, schema=None, num_rows=None, row_groups=None, key_value_metadata=None, created_by=None, column_orders=None, encryption_algorithm=None, footer_signing_key_metadata=None)
 |
 |  Description for file metadata
 |
 |  Attributes:
 |   - version: Version of this file *
 |   - schema: Parquet schema for this file.  This schema contains metadata for all the columns.
 |  The schema is represented as a tree with a single root.  The nodes of the tree
 |  are flattened to a list by doing a depth-first traversal.
 |  The column metadata contains the path in the schema for that column which can be
 |  used to map columns to nodes in the schema.
 |  The first element is the root *
 |   - num_rows: Number of rows in this file *
 |   - row_groups: Row groups in this file *
 |   - key_value_metadata: Optional key/value metadata *
 |   - created_by: String for application that wrote this file.

Можно посмотреть только `docstring` класса - так короче (нет лишней информации):

In [11]:
print(MD.__doc__)


    Description for file metadata

    Attributes:
     - version: Version of this file *
     - schema: Parquet schema for this file.  This schema contains metadata for all the columns.
    The schema is represented as a tree with a single root.  The nodes of the tree
    are flattened to a list by doing a depth-first traversal.
    The column metadata contains the path in the schema for that column which can be
    used to map columns to nodes in the schema.
    The first element is the root *
     - num_rows: Number of rows in this file *
     - row_groups: Row groups in this file *
     - key_value_metadata: Optional key/value metadata *
     - created_by: String for application that wrote this file.  This should be in the format
    <Application> version <App Version> (build <App Build Hash>).
    e.g. impala version 1.0 (build 6cf94d29b2b7115df4de2c06e2ab4326d721eb55)

     - column_orders: Sort order used for the min_value and max_value fields in the Statistics
    objects and 

# Row Groups

Данные в parquet файлы хранятся в виде последовательности "чанков" (здесь и далее буду употреблять это слоко как синоним правильному термину - "row group"), между чанками нет разделителей - они идут друг за другом. Каждый чанк может обрабатываться независимо от другого чанка.

## Page: страницы

Чанк состоит из страниц (`Page` - просто набор байт):

* для каждой колонки создается свой набор страниц
    * порядок колонок (и их страниц) определяется схемой
    * количество страниц с данными для всех колонок чанка одинаково
    * страница сжимается (поэтому страница - **минимальная единица чтения данных**, меньше прочитать нельзя)
    * страница закодирована единственным образом  
* страница всегда имеет заголовок (**PageHeader**), из которого можно понять "тип" страницы:
    * **DataPage** содержит данные (DataPageHeader)
    * **IndexPage** содержит индексную информацию (о странице данных) (IndexPageHeader)
        * редко встречается, вместо этого заполняется индексная информация на уровне колонки в чанке (см. метаданные ниже)
    * **DictionaryPage** содержит словарь (DictionaryPageHeader)
        * чанк не может содержать больше одной словарной страницы на колонку
        * как правило, словарная страница является первой в списке страниц
    * DataPage, IndexPage, DictionaryPage - это не объекты (просто названия)
        * соответствующие Header-ы - объекты, с ними познакомимся ниже
* в файле страница начинается с заголовка (PageHeader)
* за ним идет заголовок, соответствующий типу страницы (см. объекты выше)
* за ним - байты страницы как таковой
    * размер страницы содержится в PageHeader

Для того, чтобы что-либо "делать" с файлом, нужно прочитать метаданные (см. соотв. раздел файла). 

Технически это значит, что должна быть определена переменная MD

Считаем количество страниц (оно одно и то же для всех колонок)

In [12]:
grpNo = 0 # нулевая группа всегда есть

In [13]:
colNo = 1 # колонку выбираем произвольно, здесь и далее значения подобраны под "интересные" колонки конкретного файла

In [14]:
nPages = 0
if MD.row_groups[grpNo].columns[colNo].meta_data.encoding_stats: # не None если страниц больше одной
    for peStat in MD.row_groups[grpNo].columns[colNo].meta_data.encoding_stats: # статистика включает также и словарную страницу
        if peStat.page_type==0: # учитываем только страницы с данными
            nPages += peStat.count
else: # значит страница одна
    nPages = 1 

In [15]:
nPages

1

Посмотрим на страницы колонки - увидим именно страницы с данными (не None только атрибут data_page_header)

In [16]:
curOffset = MD.row_groups[grpNo].columns[colNo].meta_data.data_page_offset # смещение первой страницы
for i in range(nPages):
    ph = pqu.decodeObject(PQ_FILE,curOffset,tt.PageHeader)
    print("PAGE",i,":")
    print(pqu.getLevelStr(ph))
    curOffset += ph.compressed_page_size+pqu.getTobjSize(ph) # смещаемся на размер PageHeader-а и размер страницы (он не включает header-а)

PAGE 0 :
First level elements of <class 'ttypes.PageHeader'>
compressed_page_size: 80187
crc: None
data_page_header#
data_page_header_v2: None
dictionary_page_header: None
index_page_header: None
read()
thrift_spec#
type: 0
uncompressed_page_size: 485001
validate()
write()


### Страницы данных

В этом ноутбуке мы разбираемся с метаданными, поэтому страницы именно данных пропускаем...

### Индексные страницы

Пока не попалось примера, где бы их можно было увидеть

### Словарная страница

Чтобы посмотреть на словарную страницу - нужно найти колонку, которая использует Dictionary кодирование. Для store_sales это, например, ss_store_sk (#7)

Попробуем глянуть на PageHeader для словарной страницы: он содержит информацию только о словарной страницы, остальные атрибуты пусты (data_page_header, index_page_header)

In [17]:
curOffset = MD.row_groups[grpNo].columns[7].meta_data.dictionary_page_offset # смещение заголовка словарной страницы
dictPageHeader = pqu.decodeObject(PQ_FILE,curOffset,tt.PageHeader)
print(pqu.getLevelStr(dictPageHeader))

First level elements of <class 'ttypes.PageHeader'>
compressed_page_size: 208
crc: None
data_page_header: None
data_page_header_v2: None
dictionary_page_header#
index_page_header: None
read()
thrift_spec#
type: 2
uncompressed_page_size: 204
validate()
write()


В заголовке словарной страницы (DictionaryPageHeader) интересным является только `num_values` - количество элементов в словаре (для данной колонки в данном чанке)

In [18]:
print(pqu.getLevelStr(dictPageHeader.dictionary_page_header))

First level elements of <class 'ttypes.DictionaryPageHeader'>
encoding: 0
is_sorted: None
num_values: 51
read()
thrift_spec#
validate()
write()


# Footer - начинать нужно отсюда!

## FileMetaData

Footer содержит объект FileMetaData, прочитаем его (выше скорее всего это уже было сделано)

In [ ]:
fLen = pqu.getFooterLen(PQ_FILE)
MD = pqu.decodeObject(PQ_FILE,-(8+fLen),tt.FileMetaData,fLen)

### Озбор метаданных и начало исследования

Для просмотра объектов мета-данных можно воспользоваться функцией `getLevelStr()`: объекты хорошо сериализуются в строку "из коробки", но... решайте сами

Просто "печать" объекта `FileMetaData` - длинновато и не очень читабельно

In [ ]:
MD

In [19]:
print(pqu.getLevelStr(MD))

First level elements of <class 'ttypes.FileMetaData'>
column_orders: None
created_by: DuckDB version v1.3.2 (build 0b83e5d2f6)
encryption_algorithm: None
footer_signing_key_metadata: None
key_value_metadata: None
num_rows: 28800991
read()
row_groups[235]
schema[24]
thrift_spec#
validate()
version: 1
write()


In [20]:
pqu.showExtra(MD)

Дополнительная информация (см. также help()):

* **num_rows**: количество строк в файле
* длина списка **row_groups** говорит нам о количестве row group в файле
* **schema**: схема данных (см. ниже)
* **key_value_metadata**: список метаданных (пар ключ-значение) на уровне файла


**Что дальше**: объект FileMetaData - содержимое footer parquet файла - достаточно сложный объект, давайте разберемся с его частями.

### FileMetaData.schema: Схема табличных данных 

Схема содержится в соответствующем атрибуте-списке

* 0 элемент - корень (как правило - структура)
* остальные элементы - поля табличных данных, содержащихся в файле

(я не стал сам разбираться в схеме и Вам не рекомендую: лучше воспользоваться готовыми решениями, если только нет каких-то конкретных вопросов именно по представлению схемы данных в parquet/ У меня пока таких вопросов не было. Тип данных для каждой колонки увидим ниже)

### Метаданные row group (объект RowGroup - FileMetaData.row_groups[i])  

Метаданные файла разбиты на части по row_group-ам: для каждой группы - свои метаданные (см. список `MD.row_groups`)

In [ ]:
print(pqu.getLevelStr(MD))

В parquet файле есть как минимум одна row_group (с индексом 0):

In [21]:
print(pqu.getLevelStr(MD.row_groups[0]))

First level elements of <class 'ttypes.RowGroup'>
columns[23]
file_offset: 4
num_rows: 122880
ordinal: None
read()
sorting_columns: None
thrift_spec#
total_byte_size: 9174924
total_compressed_size: None
validate()
write()


In [ ]:
print(MD.row_groups[0].__doc__)

#### Метаданные колонки row group-ы (объект ColumnChunk - FileMetaData.row_groups[i].columns[j]) 

Для каждой колонки в метаданных есть свой раздел, содержащий объект мета-данных ColumnChunk.

В данных есть как минимум одна колонка - с индексом 0

In [22]:
print(pqu.getLevelStr(MD.row_groups[0].columns[0]))

First level elements of <class 'ttypes.ColumnChunk'>
column_index_length: None
column_index_offset: None
crypto_metadata: None
encrypted_column_metadata: None
file_offset: 0
file_path: None
meta_data#
offset_index_length: None
offset_index_offset: None
read()
thrift_spec#
validate()
write()


In [23]:
pqu.showExtra(MD.row_groups[0].columns[0])

Дополнительная информация (см. также help()):

* для каждой колонки могут быть заполнены "индексы" (не путать с метаданными колонки - объект ColumnMetaData)
    * пара атрибутов **column_index_offset/offset_index_offset** и соответствующие длины
    * "индексы" содержат мин/макс значения колонки для каждой страницы данных в пределах row_group
    * "офсеты" содержат смещение соответствующих страниц данных
    * детали - см. соответствующий раздел выше ("индексы" хранятся после страниц с данными, но до footer-а)
    * на практике заполняются достаточно редко (ибо больше одной страницы в row group также бывает редко)
* **file_offset**: смещение первой страницы (данных) колонки в пределах row_group (для первой колонки первой группы - 4, в начале файла - PAR1)
    * (deprecated) не всегда заполняется (дублирует ColumnMetaData.data_page_offset)
* **meta_data**: собственно мета-данные column_chunk, они содержатся в объекте ColumnMetaData


Что видим - "индексов" нет... Пока не попалось примера, где бы они были.

In [ ]:
print(MD.row_groups[0].columns[0].__doc__)

#### (собственно) метаданные колонки row group-ы (объект ColumnMetaData - FileMetaData.row_groups[i].columns[j].meta_data)

Финальная (нижняя по глубине вложенности) структура данных с мета-данными колонки в пределах row group

In [24]:
print(pqu.getLevelStr(MD.row_groups[0].columns[0].meta_data))

First level elements of <class 'ttypes.ColumnMetaData'>
bloom_filter_length: 4112
bloom_filter_offset: 1459762393
codec: 1
data_page_offset: 7165
dictionary_page_offset: 4
encoding_stats: None
encodings[1]
geospatial_statistics: None
index_page_offset: None
key_value_metadata: None
num_values: 122880
path_in_schema[1]
read()
size_statistics: None
statistics#
thrift_spec#
total_compressed_size: 46885
total_uncompressed_size: 53102
type: 1
validate()
write()


In [ ]:
pqu.showExtra(MD.row_groups[0].columns[0].meta_data)

In [25]:
print(pqu.getLevelStr(MD.row_groups[0].columns[0].meta_data.path_in_schema))

['ss_sold_date_sk']


Тип данных (именно в смысле "parquet"), не логический тип (см. рассуждения про схему выше)

In [26]:
tt.Type._VALUES_TO_NAMES[1]

'INT32'

In [ ]:
print(MD.row_groups[0].columns[0].meta_data.__doc__)

#### Cоставляющие объекта ColumnMetaData

##### FileMetaData.row_groups[0].columns[0].meta_data.encoding_stats: статистика используемых способов кодирования для колонки row group-ы

Не всегда бывает - если в чанке одна страница, то заполнять статистику избыточно (ее и не заполняют)

In [28]:
print(pqu.getLevelStr(MD.row_groups[0].columns[0].meta_data.encoding_stats))

First level elements of <class 'NoneType'>


Расшифровка констант - см. "Печать констант" ниже.

In [ ]:
print(pqu.getLevelStr(MD.row_groups[0].columns[0].meta_data.encodings))

In [ ]:
tt.Encoding._VALUES_TO_NAMES[8]

In [ ]:
print(tt.PageEncodingStats.__doc__)

Список содержит объекты PageEncodingStats, описывающие то, какие страницы каким образом закодированы:

* если для колонка использовалось словарное кодирование, то в списке будет более одного элемента
    * первый элемент всегда описывает страницу со словарем 
* иначе в списке будет один элемент

см. также help() выше

Из этого списка можно понять - какие способы кодирования были применены для этого column chunk.

##### FileMetaData.row_groups[0].columns[0].meta_data.statistics: статистика данных колонки row group-ы

Эта статистика хранится на уровне row_group-ы и может дополнительно присутствовать в каждой странице данных колонки в пределах row group.

Последнее как правило не используется - см. про "индексные" страницы выше, но теоретически такая возможность есть. 

То есть **на практике статистика - мин/макс/кол_уникальных/NULL - берется отсюда**

In [27]:
print(pqu.getLevelStr(MD.row_groups[0].columns[0].meta_data.statistics))

First level elements of <class 'ttypes.Statistics'>
distinct_count: 1785
is_max_value_exact: True
is_min_value_exact: True
max: b'\xa2l%\x00'
max_value: b'\xa2l%\x00'
min: b'\x80e%\x00'
min_value: b'\x80e%\x00'
null_count: 5452
read()
thrift_spec#
validate()
write()


In [ ]:
pqu.showExtra(MD.row_groups[0].columns[0].meta_data.statistics)

In [ ]:
print(MD.row_groups[0].columns[0].meta_data.statistics.__doc__)

## Печать констант

Thrift включает определение множества числовых констант (типы данных, типы кодирования, кодеки сжатия и т.п.).

Для каждого "типа" числовых констант создан соответствующий класс, в нем определен атрибут (list) `._VALUES_TO_NAMES`, с его помощью можно вывести "смысл" (строковое значение) соответствующей числовой константы.

Например, кодек сжатия:

In [ ]:
tt.Type._VALUES_TO_NAMES[1]

In [ ]:
tt.CompressionCodec._VALUES_TO_NAMES[1]

In [ ]:
tt.Encoding._VALUES_TO_NAMES[8]

Для того, чтобы узнать имя класса (в примере выше - кодека сжатия CompressionCodec) можно воспользоваться функцией `dir()`, также помним, что в юпитере работает автодополнение (TAB) - если известно начало имени класса (например, Co), то автодополнение покажет возможные продолжения. 

In [ ]:
dir(tt)